In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch.nn as nn
from torch.nn import Sequential
from torch.autograd import Variable

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from IPython.display import Markdown, display

## Load Data

In [6]:
df = pd.read_csv(
    "data/plos_fulldata.csv",
    # names=header,
    index_col=0)

/tmp/ipykernel_24224/4005757943.py:1: DtypeWarning: Columns (67,75,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [7]:
display(df)

,filename,doi_x,AccessionID,reading_time,flesch_reading_score,dale_chall_readability_score,total_paragraphs,avg_sentences_per_para,len_of_longest_paragraph,total_sentences,avg_words_per_sentence,len_of_longest_sentence,total_words,compulsive_hedgers,intensifiers,and_frequency,vague_words_count,vague_words_frequency,polysyllab_count,polysyllab_frequency,title,doi_y,pmid,pmc,ads_id,uri,altmetric_jid,issns,journal,cohorts,abstract,abstract_source,context,authors,type,altmetric_id,schema,is_oa,publisher_subjects,cited_by_fbwalls_count,cited_by_feeds_count,cited_by_gplus_count,cited_by_msm_count,cited_by_posts_count,cited_by_tweeters_count,cited_by_wikipedia_count,cited_by_accounts_count,last_updated,score,history,url,added_on,published_on,subjects,scopus_subjects,readers,readers_count,images,details_url,meta_id,isbns,handles,cited_by_rh_count,cited_by_patents_count,cited_by_rdts_count,handle,arxiv_id,cited_by_videos_count,cited_by_policies_count,cited_by_qna_count,cited_by_peer_review_sites_count,cited_by_weibo_count,cited_by_pinners_count,cited_by_linkedin_count,urn,cited_by_book_reviews_count,editors,attribution,ordinal_number
0,journal.pone.0080347.xml,10.1371/journal.pone.0080347,journal.pone.0080347,29.270,38.69,7.5,36,6.72,21,242,24.19,76,5854,34,14,3.30,38,0.006491,1141,0.194909,First Partial Skeleton of a 1.34-Million-Year-...,10.1371/journal.pone.0080347,24339873.0,PMC3855051,2013PLoSO...880347D,http://dx.plos.org/10.1371/journal.pone.0080347,4f6fa5313cf058f6100043e3,['1932-6203'],PLOS ONE,"{'pub': 61, 'sci': 15, 'doc': 1}","Recent excavations in Level 4 at BK (Bed II, O...",pubmed,"{'all': {'count': 20434347, 'mean': 9.80165359...","['Manuel Domínguez-Rodrigo', 'Travis Rayne Pic...",article,1959226,1.5.4,False,"[{'name': 'Multidisciplinary', 'scheme': 'era'}]",23.0,2.0,8.0,16.0,148,78.0,8.0,135,1632004490,207.696,"{'1y': 19.25, '6m': 3, '3m': 0, '1m': 0, '1w':...",http://dx.doi.org/10.1371/journal.pone.0080347,1386285238,1386201600,"['medicine', 'science']","['Biochemistry, Genetics and Molecular Biology...","{'citeulike': '0', 'mendeley': '145', 'connote...",145,{'small': 'https://badges.altmetric.com/?size=...,https://www.altmetric.com/details.php?citation...,10.1371/journal.pone.0080347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,journal.pone.0054670.xml,10.1371/journal.pone.0054670,journal.pone.0054670,17.565,29.11,7.3,33,4.67,13,154,22.81,73,3513,11,4,2.08,20,0.005693,801,0.228010,DNA Elements Reducing Transcriptional Gene Sil...,10.1371/journal.pone.0054670,23382937.0,NaN,2013PLoSO...854670K,NaN,4f6fa5313cf058f6100043e3,['1932-6203'],PLOS ONE,{'pub': 1},Transcriptional gene silencing (TGS)--a phenom...,pubmed,"{'all': {'count': 17963867, 'mean': 9.50228887...","['Naoki Kishimoto', 'Jun-ichi Nagai', 'Takehit...",article,1212797,1.5.4,False,"[{'name': 'Multidisciplinary', 'scheme': 'era'}]",NaN,NaN,NaN,NaN,1,1.0,NaN,1,1359728352,0.500,"{'1y': 0, '6m': 0, '3m': 0, '1m': 0, '1w': 0, ...",http://dx.doi.org/10.1371/journal.pone.0054670,1359728495,1359504000,"['medicine', 'science']","['Biochemistry, Genetics and Molecular Biology...","{'citeulike': '1', 'mendeley': '42', 'connotea...",43,{'small': 'https://badges.altmetric.com/?size=...,https://www.altmetric.com/details.php?citation...,10.1371/journal.pone.0054670,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,journal.pone.0195986.xml,10.1371/journal.pone.0195986,journal.pone.0195986,25.780,26.14,7.7,48,4.98,10,239,21.57,79,5156,25,2,4.44,93,0.018037,1267,0.245733,You pray to your God: A qualitative analysis o...,10.1371/journal.pone.0195986,29664956.0,PMC5903624,2018PLoSO..1395986A,NaN,4f6fa5313cf058f6100043e3,['1932-6203'],PLOS ONE,"{'pub': 13, 'doc': 5, 'sci': 2}","Five billion people lack access to safe, affor...",NaN,"{'all': {'count': 18780663, 'mean': 9.56645745...","['Katherine Albutt', 'Rachel R. Yorlets', 'Mar...",article,38012968,1.5.4,False,"[{'name': 'Multidisciplinary', 'scheme': 'era'}]",NaN,Na

In [14]:
pd.set_option('display.max_rows', None)
df.dtypes

filename                             object
doi_x                                object
AccessionID                          object
reading_time                        float64
flesch_reading_score                float64
dale_chall_readability_score        float64
total_paragraphs                      int64
avg_sentences_per_para              float64
len_of_longest_paragraph              int64
total_sentences                       int64
avg_words_per_sentence              float64
len_of_longest_sentence               int64
total_words                           int64
compulsive_hedgers                    int64
intensifiers                          int64
and_frequency                       float64
vague_words_count                     int64
vague_words_frequency               float64
polysyllab_count                      int64
polysyllab_frequency                float64
title                                object
doi_y                                object
pmid                            

In [114]:
df = df.fillna(0)
df = df.sample(frac=1)

In [116]:
display(df)

,filename,doi_x,AccessionID,reading_time,flesch_reading_score,dale_chall_readability_score,total_paragraphs,avg_sentences_per_para,len_of_longest_paragraph,total_sentences,...,cited_by_qna_count,cited_by_peer_review_sites_count,cited_by_weibo_count,cited_by_pinners_count,cited_by_linkedin_count,urn,cited_by_book_reviews_count,editors,attribution,ordinal_number
94505,journal.pone.0040928.xml,10.1371/journal.pone.0040928,journal.pone.0040928,23.735,26.88,7.6,36,5.47,16,197,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
83382,journal.pmed.0020151.xml,10.1371/journal.pmed.0020151,journal.pmed.0020151,6.390,12.57,9.3,20,2.50,5,50,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
90991,journal.pone.0109737.xml,10.1371/journal.pone.0109737,journal.pone.0109737,23.705,32.87,6.8,34,7.03,15,239,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
179233,journal.pntd.0002522.xml,10.1371/journal.pntd.0002522,journal.pntd.0002522,22.000,40.01,6.8,34,5.94,21,202,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
105701,journal.pone.0017867.xml,10.1371/journal.pone.0017867,journal.pone.0017867,30.485,21.26,7.7,41,6.07,13,249,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109779,journal.pone.0207944.xml,10.1371/journal.pone.0207944,journal.pone.0207944,27.480,33.58,7.2,51,5.02,25,256,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
153398,journal.pone.0086528.xml,10.1371/journal.pone.0086528,journal.pone.0086528,13.645,20.35,7.7,25,4.32,12,108,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
93636,journal.ppat.1004893.xml,10.1371/journal.ppat.1004893,journal.ppat.1004893,32.455,34.43,7.3,50,5.30,14,265,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0
84232,journal.pone.0051828.xml,10.1371/journal.pone.0051828,journal.pone.0051828,17.670,30.54,7.5,35,4.34,11,152,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0


## Multi-label --> One label

In [117]:
pd.set_option('display.max_colwidth', None)

In [ ]:
features = ['reading_time','flesch_reading_score',]

In [118]:
features = df.iloc[:, 5: 20]
feature_names = df.columns[5: 20]
labels = df[['cited_by_fbwalls_count', 'cited_by_feeds_count',
       'cited_by_gplus_count', 'cited_by_msm_count', 'cited_by_posts_count',
       'cited_by_tweeters_count', 'cited_by_wikipedia_count',
       'cited_by_accounts_count', 'score']]
features = (features - features.mean()) / features.std()
labels = (labels - labels.mean()) / labels.std()
labels = labels.sum(axis=1)

In [119]:
train_set, validation_set, test_set = np.split(features, [int(0.6 * len(features)), int(0.8 * len(features))])
train_label, validation_label, test_label = np.split(labels, [int(0.6 * len(features)), int(0.8 * len(features))])

### Random Forest Regressor

In [120]:
model = RandomForestRegressor(max_depth=10)
model.fit(train_set, train_label)

RandomForestRegressor(max_depth=10)

In [121]:
score = model.score(train_set, train_label)
print(score)

0.4428386272018838


In [122]:
score = model.score(validation_set, validation_label)
print(score)

0.03363215743585468


### DecisionTreeRegressor

### K Nearest

## One label

In [139]:
features = df.iloc[:, 5: 20]
feature_names = df.columns[5: 20]
labels1 = df[['score']]
features = (features - features.mean()) / features.std()
labels1 = (labels1 - labels1.mean()) / labels1.std()
labels1 = labels1.sum(axis=1)

In [140]:
train_set, validation_set, test_set = np.split(features, [int(0.6 * len(features)), int(0.8 * len(features))])
train_label, validation_label, test_label = np.split(labels, [int(0.6 * len(features)), int(0.8 * len(features))])

### Random Forest Regressor

In [141]:
model = RandomForestRegressor(max_depth=10)
model.fit(train_set, train_label)

RandomForestRegressor(max_depth=10)

In [142]:
score = model.score(train_set, train_label)
print(score)

0.437116745265817


In [143]:
score = model.score(validation_set, validation_label)
print(score)

0.02275082578182186


### DecisionTreeRegressor

### K Nearest